In [ ]:
import requests
import time
import json
import csv

# Define constants
API_KEY = 'YOUR_ETHERSCAN_API_KEY'  # Replace with your Etherscan API key
WALLET_ADDRESS = '0xd4bC53434C5e12cb41381A556c3c47e1a86e80E3'
ETHERSCAN_API_URL = "https://api.etherscan.io/api"
START_BLOCK = 0  # Starting block (could be set to a higher value to narrow down the search)
END_BLOCK = 99999999  # Use a high block number for the latest block

OFFSET_LENGTH = 2500

def get_erc20_transfers(wallet_address, start_block=START_BLOCK, end_block=END_BLOCK):
    transfers = []
    page = 1  # Start with page 1
    while True:
        # Call Etherscan API to get ERC-20 token transfer events
        params = {
            'module': 'account',
            'action': 'tokentx',
            'address': wallet_address,
            'startblock': start_block,
            'endblock': end_block,
            'page': page,
            'offset': OFFSET_LENGTH,  # Number of transactions per page
            'sort': 'desc',  # Sort by ascending block number
            'apikey': API_KEY
        }

        response = requests.get(ETHERSCAN_API_URL, params=params)
        data = response.json()

        if data['status'] == '1' and 'result' in data:
            transfers.extend(data['result'])
            print(len(transfers))

            # If there are more results, move to the next page
            if len(data['result']) == OFFSET_LENGTH:
                page += 1
            else:
                break  # All results retrieved
        else:
            print(f"Error: {data.get('message', 'Unknown error')}")
            break

        # Respect rate limits (about 5 requests per second)
        time.sleep(1)

    return transfers

def save_transfers_to_csv(transfers, filename='erc20_transfers.csv'):
    # Define the CSV header
    header = ['BlockNumber', 'TimeStamp', 'Hash', 'From', 'To', 'ContractAddress', 'TokenName', 'TokenSymbol', 'Value', 'Gas', 'GasPrice', 'IsError', 'TxReceiptStatus']
    
    # Open CSV file for writing
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()  # Write the header row
        for transfer in transfers:
            writer.writerow({
                'BlockNumber': transfer.get('blockNumber'),
                'TimeStamp': transfer.get('timeStamp'),
                'Hash': transfer.get('hash'),
                'From': transfer.get('from'),
                'To': transfer.get('to'),
                'ContractAddress': transfer.get('contractAddress'),
                'TokenName': transfer.get('tokenName'),
                'TokenSymbol': transfer.get('tokenSymbol'),
                'Value': transfer.get('value'),
                'Gas': transfer.get('gas'),
                'GasPrice': transfer.get('gasPrice'),
                'IsError': transfer.get('isError'),
                'TxReceiptStatus': transfer.get('txReceiptStatus')
            })

    print(f"Data saved to {filename}")

# Fetch ERC-20 token transfers
transfers = get_erc20_transfers(WALLET_ADDRESS)

# Print the results and save them to a CSV file
if transfers:
    print(f"Found {len(transfers)} ERC-20 token transfers for wallet {WALLET_ADDRESS}")
    save_transfers_to_csv(transfers)
else:
    print("No ERC-20 transfers found.")

2500


In [ ]:
import pandas as pd

# Load the CSV file containing the ERC-20 token transfers
file_path = 'erc20_transfers.csv'  # Make sure the path is correct if not in the same directory
df_erc20 = pd.read_csv(file_path, index_col='Hash')

# Convert 'TimeStamp' from Unix timestamp to yyyy-mm-dd hh:mm:ss format
df_erc20['TimeStamp'] = pd.to_datetime(df_erc20['TimeStamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M')

# Rename 'Value' to 'Value_ETH' to represent Ether values
df_erc20.rename(columns={'Value': 'Value_Eth'}, inplace=True)
df_erc20.rename(columns={'GasPrice': 'GasPrice_Gwei'}, inplace=True)

# Convert 'Value_ETH' from Wei to Ether 
df_erc20['Value_Eth'] = pd.to_numeric(df_erc20['Value_Eth'], errors='coerce') / 10**18  # Convert Wei to Ether

# Convert GasPrice from Wei to Gwei and Ether
df_erc20['GasPrice_Gwei'] = pd.to_numeric(df_erc20['GasPrice_Gwei'], errors='coerce') / 10**9

# # Convert GasPrice_Gwei to Ether
# df_erc20['GasPrice_Eth'] = df_erc20['GasPrice_Gwei'] * 10**-9

# Calculate Transaction Cost in Ether
df_erc20['TransactionCost_Eth'] = df_erc20['Gas'] * df_erc20['GasPrice_Gwei'] * 10**-9 * 0.7

# Save the modified DataFrame to the same CSV file
df_erc20.to_csv('./csv/erc20_transfers.csv')

In [ ]:
df_erc20